# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [112]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from connect import *

In [113]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

8057


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [136]:
df = pd.read_csv('/home/ahmed/work/courses/Data Engineering Nanodegree/projects/Data-Engineer-Nanodegree/Data Modeling/Data Modeling using Cassandra Database/event_datafile_new.csv')
df.head(4)

,Unnamed: 0,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,0,Great Lake Swimmers,Logged In,Kevin,M,0,Arellano,215.11791,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540010e+12,815,Your Rocky Spine,200,1.542930e+12,66.0
1,1,Soziedad Alkoholika,Logged In,Kevin,M,1,Arellano,204.74730,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540010e+12,815,Va Bien,200,1.542930e+12,66.0
2,2,Franz Ferdinand,Logged In,Kevin,M,2,Arellano,172.01587,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540010e+12,815,Eleanor Put Your Boots On,200,1.542930e+12,66.0
3,3,Modest Mouse,Logged In,Kevin,M,3,Arellano,209.52771,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1.540010e+12,815,Float On,200,1.542930e+12,66.0


In [137]:
df.columns

Index(['Unnamed: 0', 'artist', 'auth', 'firstName', 'gender', 'itemInSession',
       'lastName', 'length', 'level', 'location', 'method', 'page',
       'registration', 'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')

In [138]:
df.drop('Unnamed: 0' , axis=1 , inplace = True)
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [139]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

#  To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [140]:
# TO-DO: Create a Keyspace 
session.execute("CREATE KEYSPACE IF NOT EXISTS cassandra WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

#### Set Keyspace

In [141]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('cassandra')

In [142]:
## list all Keyspace from cassandra
keyspaces = session.execute("SELECT * FROM system_schema.keyspaces;")
for keyspace in keyspaces:
    print(keyspace)

Row(keyspace_name='test', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='system_auth', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='system_schema', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.LocalStrategy')]))
Row(keyspace_name='cassandra', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='tutorialspoint', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.locator.SimpleStrategy'), ('replication_factor', '1')]))
Row(keyspace_name='system_distributed', durable_writes=True, replication=OrderedMapSerializedKey([('class', 'org.apache.cassandra.loca

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [143]:
print(df.columns)

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8056 entries, 0 to 8055
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   auth           8056 non-null   object 
 2   firstName      7770 non-null   object 
 3   gender         7770 non-null   object 
 4   itemInSession  8056 non-null   int64  
 5   lastName       7770 non-null   object 
 6   length         6820 non-null   float64
 7   level          8056 non-null   object 
 8   location       7770 non-null   object 
 9   method         8056 non-null   object 
 10  page           8056 non-null   object 
 11  registration   7770 non-null   float64
 12  sessionId      8056 non-null   int64  
 13  song           6820 non-null   object 
 14  status         8056 non-null   int64  
 15  ts             8056 non-null   float64
 16  userId         7770 non-null   float64
dtypes: float64(4), int64(3), object(10)
memory usage: 1.

In [168]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

"""
    CreateTable Function 
    Input : 
            Table Name 
            Fields
    output :
            return query        
"""
def createTable(nameTable , fields):
    query = "CREATE TABLE IF NOT EXISTS {}".format(nameTable)
    query = query + fields
    return query

"""
    insert Values Function 
    Input : 
            dataFrame
            Fields
    output :
            no output         
"""
def insertValues(df , tableName , fields , session):
    for index , row in df.iterrows():
        print(row)
        query1 = "INSERT INTO  " + str(tableName) + " {}".format(fields)
        query1 = query1 + " VALUES (%s, %s, %s ,%s , %s)"
        session.execute(query1 , (row[x]for x in df.columns))
        


    

"""
    Execute Query Function
    input : 
            session
            query
    output : no output it only execute query
"""
def executeQuery(session , query):
    session.execute(query)


"""
    drop table  Function
    input : 
            session
            Table Name

    output : no output it only execute query
"""

def dropTable(session , tableName):
    session.execute("drop table {}".format(tableName))



In [147]:

tableName  = "musicApp"
fields = "(artist text , song text , length float , sessionId bigint , itemInSession int, PRIMARY KEY (sessionId , itemInSession))"
query = createTable(tableName , fields)
## create table in database
executeQuery(session , query)

In [148]:
## insert values in cassandra
field = "(artist , song  , length  , sessionId  , itemInSession )"
df_1 = df[['artist' , 'song'  , 'length'  , 'sessionId'  , 'itemInSession']]
df_1.dropna(axis = 0 , inplace=True)
# print(df)
insertValues(df_1 , tableName , field , session)

/home/ahmed/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


#### Do a SELECT to verify that the data have been inserted into each table

In [149]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows =  session.execute("select * from musicApp where sessionId = 338 and itemInSession = 4")
for row in rows:
    print (row.artist , row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [176]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userId'],
      dtype='object')

In [184]:
dropTable(session , "musicApp2")

In [185]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
tableName = "musicApp2"
fields = "(userid bigint, sessionid bigint ,itemInSession int ,artist text , song text , firstName text, lastName  text , PRIMARY KEY ((userid , sessionid) ,itemInSession ))"
df_2 = df[['userId' , 'sessionId' , 'itemInSession','artist' , 'song'  , 'firstName'  , 'lastName']]
query = createTable(tableName , fields)
## create table in database
executeQuery(session , query)

df_2.dropna(axis = 0 , inplace=True)                    

/home/ahmed/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [186]:
df_2.head(3)

,userId,sessionId,itemInSession,artist,song,firstName,lastName
0,66.0,815,0,Great Lake Swimmers,Your Rocky Spine,Kevin,Arellano
1,66.0,815,1,Soziedad Alkoholika,Va Bien,Kevin,Arellano
2,66.0,815,2,Franz Ferdinand,Eleanor Put Your Boots On,Kevin,Arellano


In [187]:
df_2.columns

Index(['userId', 'sessionId', 'itemInSession', 'artist', 'song', 'firstName',
       'lastName'],
      dtype='object')

In [188]:
field = "(userId , sessionId , itemInSession ,artist , song , firstName , lastName )"
insertValues(df_2 , tableName , field , session)

userId                          66.0
sessionId                        815
itemInSession                      0
artist           Great Lake Swimmers
song                Your Rocky Spine
firstName                      Kevin
lastName                    Arellano
Name: 0, dtype: object


TypeError: not all arguments converted during string formatting

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()